In [1]:
import pandas as pd
import numpy as np
import pytz 
import datetime

# def convert_to_utc(time_zone_string, naive_datetime, is_dst_bool=None):
#     # print is_dst_bool
#     local_timezone = pytz.timezone(time_zone_string)
#     local_dt = local_timezone.localize(naive_datetime, is_dst=is_dst_bool)
#     utc_dt = local_dt.astimezone(pytz.utc)
#     return utc_dt

# def convert_to_local(time_zone_string, naive_datetime, is_dst_bool=None):
#     # print is_dst_bool
#     local_timezone = pytz.timezone(time_zone_string)
#     local_dt = local_timezone.localize(naive_datetime, is_dst=is_dst_bool)
#     # utc_dt = local_dt.astimezone(pytz.utc)
#     return local_dt

def convert_to_local_timestamp(time_zone_string, naive_timestamp, is_dst_bool=None):
    # print is_dst_bool
    local_timezone = pytz.timezone(time_zone_string)
    naive_datetime = datetime.datetime(naive_timestamp.year,naive_timestamp.month,naive_timestamp.day,naive_timestamp.hour,naive_timestamp.minute,naive_timestamp.second)
    local_dt = local_timezone.localize(naive_datetime, is_dst=is_dst_bool)
    # utc_dt = local_dt.astimezone(pytz.utc)
    return local_dt

def convert_to_utc_timestamp(time_zone_string, naive_timestamp, is_dst_bool=None):
    # print is_dst_bool
    local_timezone = pytz.timezone(time_zone_string)
    naive_datetime = datetime.datetime(naive_timestamp.year,naive_timestamp.month,naive_timestamp.day,naive_timestamp.hour,naive_timestamp.minute,naive_timestamp.second)
    local_dt = local_timezone.localize(naive_datetime, is_dst=is_dst_bool)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt

In [2]:
base_path='C:/Shaukat/code/data_rep/bom/historical_data/2013-2016/TAS/'
filename='HM01X_Data_094029_999999999405613'
ext='.txt'
path_to_file=base_path+filename+ext
df = pd.read_csv(path_to_file)



In [3]:
# df.to_csv('Data_091107.csv')

In [4]:
# ADD UTC and Local Time
timezone_string='Australia/Hobart'
df['naive_mmddyyyy'] = df.apply(lambda row: datetime.datetime(row['Year Month Day Hour Minutes in YYYY'], row['MM'], row['DD'], row['HH24'],0,0), axis=1)
df['local_mmddyyyy'] = df['naive_mmddyyyy'].apply(lambda row: convert_to_local_timestamp(timezone_string, row, True))
df['utc_mmddyyyy'] = df['local_mmddyyyy'].apply(lambda row: convert_to_utc_timestamp(timezone_string, row, True))


In [5]:
# ADD Rainfall accumulation

# Converting columns into float
df['Precipitation since 9am local time in mm'] =  pd.to_numeric(df['Precipitation since 9am local time in mm'], errors='coerce')

# Appending past values in a separate column
df['precipitaion_lasthour'] = df['Precipitation since 9am local time in mm'].shift(1)
# Calculating rainfall since last hour
df['rainfall_since_last_hour'] = df['Precipitation since 9am local time in mm'] - df['precipitaion_lasthour']

# Modifying all negative values in a column by 0 (This shows that rainfall next hour is either taken from other station
# or the value is reset at 00:00)
df.loc[df['rainfall_since_last_hour'] < 0, 'rainfall_since_last_hour'] = 0.0 



In [6]:
del base_path, path_to_file
base_path='C:/Shaukat/code/data_rep/bom/historical_data/2013-2016/TAS/csv/'
path_to_file=base_path+filename+'.csv'
df.to_csv(path_to_file)